In [1]:
import numpy as np
import glob
import cv2 as img
import scipy.io

In [2]:
def Load_Data(path):
    trainx = []
    trainy = []
    testx = []
    testy = []
    for i in range(10):
        for filename in glob.glob(path+'\\train\\'+str(i)+'\\*.png'):
            im = img.imread(filename)
            trainx.append(im)
            trainy.append(i)
    for i in range(10):
        for filename in glob.glob(path+'\\test\\'+str(i)+'\\*.png'):
            im = img.imread(filename)
            testx.append(im)
            testy.append(i)
    print("Data Loaded")
    return np.array(trainx),np.array(trainy),np.array(testx),np.array(testy)

In [3]:
trainx,trainy,testx,testy = Load_Data("C:\\Users\\Tauseef Chahal\\DeepLearning_Code\\Deep\\Assignment2\\Task3_Data\\")

Data Loaded


In [7]:
mat = scipy.io.loadmat('filters.mat')

In [8]:
mat = np.array(mat['filters'])
print(mat.shape)

(17, 17, 36)


In [9]:
print(mat[0].shape[-1])

36


In [10]:
print(trainx.shape)

(60000, 28, 28, 3)


In [11]:
def cnn_layer(X,features):
    #how big is our filter/patch?
    nb_features = features[0].shape[-1]
    #how many features do we have?
    patch_dim = features.shape[0]
    #How big is our image?
    image_dim = X.shape[2] #28x28
    #R G B values
    image_channels = X.shape[3]#3
    #how many images do we have?
    nb_images = X.shape[0]#6000 trainx

    conv_dim = image_dim - patch_dim + 1
    
    convolved_features = np.zeros((nb_images, nb_features, conv_dim, conv_dim));
    #then we'll iterate through each image that we have
    for image_i in range(nb_images):
        #for each feature 
        for feature_i in range(nb_features):
            #lets initialize a convolved image as empty
            convolved_image = np.zeros((conv_dim, conv_dim))
            #then for each channel (r g b )
            for channel in range(image_channels):
                #lets extract a feature from our feature map
                feature = features[:, :,feature_i]
                #then define a channel specific part of our image
                image   = X[image_i, :, :, channel]
                #perform convolution on our image, using a given feature filter
                convolved_image = convolved_image + convolve2d(image, feature);

            convolved_features[image_i, feature_i, :, :] = convolved_image
    return convolved_features

In [12]:
def convolve2d(image, feature):
    image_dim = np.array(image.shape)
    feature_dim = np.array(feature.shape)
    target_dim = image_dim - feature_dim + 1
    fft_result = np.fft.fft2(image, target_dim) * np.fft.fft2(feature, target_dim)
    target = np.fft.ifft2(fft_result).real
    return target

In [13]:
a = cnn_layer(trainx,mat)

KeyboardInterrupt: 

In [42]:
print(a.shape)

(60000, 36, 12, 12)


In [ ]:
def maxpooling_layer(self, convolved_features):
    #given our learned features and images
    nb_features = convolved_features.shape[0]
    nb_images = convolved_features.shape[1]
    conv_dim = convolved_features.shape[2]
    res_dim = int(conv_dim / self.pool_size)       #assumed square shape

    #initialize our more dense feature list as empty
    pooled_features = np.zeros((nb_features, nb_images, res_dim, res_dim))
    #for each image
    for image_i in range(nb_images):
        #and each feature map
        for feature_i in range(nb_features):
            #begin by the row
            for pool_row in range(res_dim):
                #define start and end points
                row_start = pool_row * self.pool_size
                row_end   = row_start + self.pool_size

                #for each column (so its a 2D iteration)
                for pool_col in range(res_dim):
                    #define start and end points
                    col_start = pool_col * self.pool_size
                    col_end   = col_start + self.pool_size

                    #define a patch given our defined starting ending points
                    patch = convolved_features[feature_i, image_i, row_start : row_end,col_start : col_end]
                    #then take the max value from that patch
                    #store it. this is our new learned feature/filter
                    pooled_features[feature_i, image_i, pool_row, pool_col] = np.max(patch)
    return pooled_features

In [ ]:
def flatten_layer(X):
    flatX = np.zeros((X.shape[0],np.prod(X.shape[1:])))
    for i in range(X.shape[0]):
        flatX[i,:] = X[i].flatten(order='C')
    return flatX

In [43]:
NN_ARCHITECTURE = [
    {"input_dim": 2, "output_dim": 25, "activation": "relu"},
    {"input_dim": 25, "output_dim": 50, "activation": "relu"},
    {"input_dim": 50, "output_dim": 50, "activation": "relu"},
    {"input_dim": 50, "output_dim": 25, "activation": "relu"},
    {"input_dim": 25, "output_dim": 1, "activation": "sigmoid"},
]

In [44]:
def init_layers(nn_architecture, seed = 99):
    # random seed initiation
    np.random.seed(seed)
    # number of layers in our neural network
    number_of_layers = len(nn_architecture)
    # parameters storage initiation
    params_values = {}
    
    # iteration over network layers
    for idx, layer in enumerate(nn_architecture):
        # we number network layers from 1
        layer_idx = idx + 1
        
        # extracting the number of units in layers
        layer_input_size = layer["input_dim"]
        layer_output_size = layer["output_dim"]
        
        # initiating the values of the W matrix
        # and vector b for subsequent layers
        params_values['W' + str(layer_idx)] = np.random.randn(
            layer_input_size + 1, layer_output_size) * 0.1
        
    return params_values

In [45]:
def sigmoid(x):
    return 1/(1+np.exp(-x))
def relu(x):
    return np.maximum(0,x)

def sigmoid_backward(dA, x):
    sig = sigmoid(x)
    return dA * sig * (1 - sig)

def relu_backward(dA, x):
    dZ = np.array(dA, copy = True)
    dZ[Z <= 0] = 0;
    return dZ;
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)
def softmax_backward(x):
    

In [46]:
def single_layer_forward_propagation(A_prev, W_curr, activation="relu"):
    # calculation of the input value for the activation function
    Ws = W_curr.T
    c = len(A_prev[0])
    A_prev = np.insert(A_prev,0,np.ones(c),axis = 0)    
    Z_curr = np.dot(Ws,A_prev)
    
    # selection of activation function
    if activation is "relu":
        activation_func = relu
    elif activation is "sigmoid":
        activation_func = sigmoid
    else:
        raise Exception('Non-supported activation function')
    # return of calculated activation A and the intermediate Z matrix
    return activation_func(Z_curr), Z_curr

In [47]:
def full_forward_propagation(X, params_values, nn_architecture):
    # creating a temporary memory to store the information needed for a backward step
    memory = {}
    # X vector is the activation for layer 0 
    A_curr = X
    
    # iteration over network layers
    for idx, layer in enumerate(nn_architecture):
        # we number network layers from 1
        layer_idx = idx + 1
        # transfer the activation from the previous iteration
        A_prev = A_curr
        
        # extraction of the activation function for the current layer
        activ_function_curr = layer["activation"]
        # extraction of W for the current layer
        W_curr = params_values["W" + str(layer_idx)]
        # extraction of b for the current layer
        #b_curr = params_values["b" + str(layer_idx)]
        # calculation of activation for the current layer
        A_curr, Z_curr = single_layer_forward_propagation(A_prev, W_curr,activ_function_curr)
        
        # saving calculated values in the memory
        memory["A" + str(idx)] = A_prev
        memory["Z" + str(layer_idx)] = Z_curr
       
    # return of prediction vector and a dictionary containing intermediate values
    return A_curr, memory

In [48]:
def get_cost_value(Y_hat, Y):
    # number of examples
    m = Y_hat.shape[1]
    # calculation of the cost according to the formula
    cost = -1 / m * (np.dot(Y, np.log(Y_hat).T) + np.dot(1 - Y, np.log(1 - Y_hat).T))
    return np.squeeze(cost)

In [50]:
# an auxiliary function that converts probability into class
def convert_prob_into_class(probs):
    probs_ = np.copy(probs)
    probs_[probs_ > 0.5] = 1
    probs_[probs_ <= 0.5] = 0
    return probs_

In [51]:
def get_accuracy_value(Y_hat, Y):
    Y_hat_ = convert_prob_into_class(Y_hat)
    return (Y_hat_ == Y).all(axis=0).mean()

In [52]:
def single_layer_backward_propagation(dA_curr, W_curr, Z_curr, A_prev, activation="relu"):
    # number of examples
    m = A_prev.shape[1]
    
    # selection of activation function
    if activation is "relu":
        backward_activation_func = relu_backward
    elif activation is "sigmoid":
        backward_activation_func = sigmoid_backward
    else:
        raise Exception('Non-supported activation function')
    
    # calculation of the activation function derivative
    dZ_curr = backward_activation_func(dA_curr, Z_curr)
    
    # derivative of the matrix W
    c = len(A_prev[0])
    A_prev = np.insert(A_prev,0,np.ones(c),axis = 0)
    dW_curr = np.dot(dZ_curr, A_prev.T) / m
    
    # derivative of the vector b
#    db_curr = np.sum(dZ_curr, axis=1, keepdims=True) / m
    # derivative of the matrix A_prev
    dA_prev = np.dot(W_curr[1:], dZ_curr)
    
    return dA_prev, dW_curr.T

In [53]:
def full_backward_propagation(Y_hat, Y, memory, params_values, nn_architecture):
    grads_values = {}
    
    # number of examples
    m = Y.shape[1]
    # a hack ensuring the same shape of the prediction vector and labels vector
    Y = Y.reshape(Y_hat.shape)
    
    # initiation of gradient descent algorithm
    dA_prev = - (np.divide(Y, Y_hat) - np.divide(1 - Y, 1 - Y_hat));
    
    for layer_idx_prev, layer in reversed(list(enumerate(nn_architecture))):
        # we number network layers from 1
        layer_idx_curr = layer_idx_prev + 1
        # extraction of the activation function for the current layer
        activ_function_curr = layer["activation"]
        
        dA_curr = dA_prev
      
        A_prev = memory["A" + str(layer_idx_prev)]
        Z_curr = memory["Z" + str(layer_idx_curr)]
        
        W_curr = params_values["W" + str(layer_idx_curr)]
        
        dA_prev, dW_curr = single_layer_backward_propagation(dA_curr, W_curr, Z_curr, A_prev, activ_function_curr)
        
        grads_values["dW" + str(layer_idx_curr)] = dW_curr
    
    return grads_values

In [54]:
def update(params_values, grads_values, nn_architecture, learning_rate):

    # iteration over network layers
    for layer_idx, layer in enumerate(nn_architecture, 1):
        params_values["W" + str(layer_idx)] -= learning_rate * grads_values["dW" + str(layer_idx)]        
    return params_values;

In [55]:
def train(X, Y, nn_architecture, epochs, learning_rate, verbose=False, callback=None):
    # initiation of neural net parameters
    params_values = init_layers(nn_architecture, 2)
    # initiation of lists storing the history 
    # of metrics calculated during the learning process 
    cost_history = []
    accuracy_history = []
    
    # performing calculations for subsequent iterations
    for i in range(epochs):
        # step forward
        Y_hat, cashe = full_forward_propagation(X, params_values, nn_architecture)
        
        # calculating metrics and saving them in history
        cost = get_cost_value(Y_hat, Y)
        cost_history.append(cost)
        accuracy = get_accuracy_value(Y_hat, Y)
        accuracy_history.append(accuracy)
        
        # step backward - calculating gradient
        grads_values = full_backward_propagation(Y_hat, Y, cashe, params_values, nn_architecture)
        # updating model state
        params_values = update(params_values, grads_values, nn_architecture, learning_rate)
        
        if(i % 50 == 0):
            if(verbose):
                print("Iteration: {:05} - cost: {:.5f} - accuracy: {:.5f}".format(i, cost, accuracy))
            if(callback is not None):
                callback(i, params_values)
            
    return params_values

In [ ]:
train_x = train_x.reshape(train_x.shape[0], 28, 28, 1)
test_x = test_x.reshape(test_x.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)
# Making sure that the values are float so that we can get decimal points after division
train_x = train_x.astype('float32')
test_x = test_x.astype('float32')
# Normalizing the RGB codes by dividing it to the max RGB value.
train_x /= 255
test_x /= 255
print('x_train shape:', train_x.shape)
print('Number of images in x_train', train_x.shape[0])
print('Number of images in x_test', test_x.shape[0])

def relu(feature_map):
    #Preparing the output of the ReLU activation function.
    m , filter_dim,img_dim , img_dim =feature_map.shape
    relu = np.zeros(feature_map.shape)
    for i in range(feature_map.shape[-1]):
        for j in np.arange(0,m):
            for k in np.arange(0, filter_dim):
                relu_out[j, k, i] = np.max([feature_map[j, k, i], 0])
    return relu_out